# <center> Laboratorio 1 </center>
## <center> Computación científica II </center>
## <center> Ariel Sanhueza Román - Gonzalo Moya Rodríguez </center>



# Previo
Primero, importaremos las bibliotecas previas:

In [4]:
import numpy as np
from scipy import linalg
%matplotlib inline
from matplotlib import pyplot as plt
from numpy.linalg import norm, solve

Ahora, cargaremos los datos entregados y generamos el vector inicial:

In [42]:
cov_x = np.load('arcene.npy')
cov_x -= cov_x.mean(axis=0)
cov_x = np.dot(np.transpose(cov_x),cov_x)
cov_x = np.divide(cov_x,n-1)
n = cov_x.shape[1]
i_guess  = np.ones(cov_x.shape[0])

# Desarrollo

## Pregunta 1
Primero, implementaremos los algoritmos de $\textbf{Power Iteration}$ y $\textbf{Rayleigh Quotient Iteration}$

### Power Iteration

In [44]:
# Adaptación del algoritmo publicado en el ipython de clases
def powerit(A, x, k):
  """
  Program 12.1 Power iteration
  Computes dominant eigenvector of square matrix
  Input: matrix A, initial (nonzero) vector x, number of steps k
  Output: dominant eigenvalue lam, eigenvector u
  """
  for j in range(k):
    u = np.divide(x, linalg.norm(x))
    x = np.dot(A, u)
    lam = float(np.dot(u.T, x))
  u = np.divide(x, linalg.norm(x))
  return lam, u

Dentro de todo el algoritmo, las líneas más costosas son (con sus respectivas complejidades):
* np.dot(A,U) $\rightarrow O(n^2)$, pues es un producto matriz vector.
* np.dot(u.T, x) $\rightarrow O(n)$, pues es un producto vector vector.
* El loop for, que depende de k.

Por lo que la complejidad de este algoritmo es de orden $O(kn^2)$

### Rayleigh Quotient Iteration

In [43]:
# Adaptación del algoritmo publicando en el ipython de clases
def rqi(A, x, k):
  """
  Program 12.3 Rayleigh Quotient Iteration
  Input: matrix A, initial (nonzero) vector x, number of steps k
  Output: eigenvalue lam, eigenvector of inv(A-sI)
  """
  for j in range(k):
    u = np.divide(x, linalg.norm(x))
    lam = float(np.dot(u.T, np.dot(A, u)))
    x = np.linalg.solve(A -lam*np.eye(*A.shape), u)
  u = np.divide(x, linalg.norm(x))
  lam = float(np.dot(u.T, np.dot(A, u)))
  return lam, u

Dentro de todo el algoritmo, las líneas más costosas son (con sus respectivas complejidades):
* np.dot(u.T, np.dot(A, u)) $\rightarrow O(n^2)$, pues está compuesto por dos productos puntos, por lo que la línea completa son aproximadamente $n^2 + n$ operaciones, lo que asintóticamente es $O(n^2)$
* El loop for, que depende de k
Por lo que la complejidad de este algoritmo es de orden $O(kn^2)$

## Pregunta 2

### Parte a)
Primero, como $A$ es simétrica, tenemos que sus valores propios son reales y sus vectores propios son ortogonales entre sí.

Tomemos $B = A - \lambda_iv_iv^T_i$. Entonces:

\begin{align*}
    Bv_k &= \lambda_kv_k \\
    (A - \lambda_iv_iv^T_i)v_k &= \lambda_kv_k \\
    Av_k - \lambda_iv_iv^T_iv_k &= \lambda_kv_k,\texttt{ pero }v^T_iv_k = 0 \\
    Av_k &= \lambda_kv_k
\end{align*}

Entonces vemos que los vectores y valores propios son los mismos en A y en B.

Ahora supongamos que $v_1$ es el vector propio dominante y $\lambda_1$ es el valor propio dominante de A (encontrado en la primera iteración, pues el valor y vector propio inicial son cero). 

\begin{align*}
    Bv_i &= \lambda_iv_i \\
    (A - \lambda_1v_1v^T_1)v_i &= \lambda_iv_i \\
    Av_i - \lambda_1v_1v^T_1v_i &= \lambda_iv_i, \texttt{ pero para } i \neq 1 \rightarrow v^T_1v_i = 0 \\
    Av_i &= \lambda_iv_i
\end{align*}

Entonces, se tienen que si aplicamos power iteration en B, encontraremos su valor/vector propio dominante. Pero como los vectores son los mismos en A y en B y $v_1$ y $\lambda_1$ no son dominantes en B, entonces los dominantes en B son los segundos más grandes en A. Si aplicamos esto $k$ veces, obtenemos los $k$ vectores y valores propios dominantes de A

### Parte b)
La complejidad de Power iteration vimos que era $O(pn^2)$ (en el caso del algoritmo propuesto, en power iteration $k = p$). Como el power iteration es ejecutado $k$ veces, entonces la complejidad total es $O(kpn^2)$.

### Parte c)

In [23]:
def k_eigen_finder(A, p, k, v_0):
    v = v_0
    l = 0
    lambdas = []
    vectors = []
    for i in range(k):
        A = A - l*np.dot(v, np.transpose(v))
        l, v = powerit(A, np.ones((A.shape[0],1)), p)
        lambdas.append(l)
        vectors.append(v)
    return lambdas, vectors

## Pregunta 3
### Parte a)

### Parte b)

### Parte c)

In [8]:
def k_eigen_finder_plus(A, p, k, v_0):
    A_p = np.linalg.matrix_power(A, p)
    l = 0
    v = np.zeros((A.shape[0], 1))
    x = v_0
    lambdas = []
    vectors = []
    for i in range(k):
        A_p = A_p - (l**p)*np.dot(v, np.transpose(v))
        v = np.dot(A_p,x)
        v = np.divide(v, linalg.norm(v))
        l = float(np.dot(v.T, np.dot(A, v)))
        lambdas.append(l)
        vectors.append(v)
    return lambdas, vectors

## Pregunta 4
### Parte a)
Las modificaciones son pocas y simples:
* Si nuestra mariz es de dimensiones $nxn$, nuestro Q_0 inicial es cualquier matriz de dimensiones $nxk$, donde $k$ es la cantidad de eigen values que queremos encontrar.
* Al realizar la descomposición QR, usaremos su versión reducida.

### Parte b)
La complejidad de este nuevo algoritmo está determinada por:
* La factorización QR, que es de complejidad $O(n^3)$.
* La cantidad de iteraciones realizadas por el Unshifted QR (llamémoslo $m$).
Por lo tanto, la complejidad total es de $O(mn^3)$.

### Parte c)
La implementación es la siguiente forma:

In [28]:
def unshifted_qr_k(A, m, k):
    # The intial value of Q
    Q = np.dot(A, np.ones((A.shape[0], k)))
    for i in range(m):
        Q,_ = np.linalg.qr(np.dot(A,Q), mode='reduced')
    return np.diagonal(np.dot(np.dot(Q.T, A), Q))

## Pregunta 5

In [46]:
lam, u = powerit(cov_x, i_guess, 27)
print lam
print u

1838214.89054
[  7.92615633e-03  -1.03486901e-04  -1.64245597e-04 ...,   1.10023573e-02
  -2.38882889e-02   4.63108496e-05]


In [45]:
lam, u = rqi(cov_x, i_guess, 3)
print lam
print u

68521.5351152
[-0.02358317 -0.00066283 -0.0007175  ...,  0.03846556 -0.05225003
 -0.00013062]


In [47]:
timeit powerit(cov_x, i_guess, 27)

10 loops, best of 3: 93.1 ms per loop


In [48]:
timeit rqi(cov_x, i_guess, 3)

1 loops, best of 3: 23.2 s per loop


In [54]:
memit powerit(conv_x, i_guess, 27)

SyntaxError: invalid syntax (<ipython-input-54-fad12471982a>, line 1)

## Pregunta 6

In [32]:
l,u = k_eigen_finder(cov_x, 100, 10, i_guess)
print l

[1838214.8906145599, 1195796.4301467692, 214003.16585597023, 154145.59217659797, 102040.01804763047, 68731.92840621894, 53790.199586216244, 49414.41073076937, 49692.74877308696, 39916.31292660358]


In [30]:
unshifted_qr_k(cov_x, 100, 10)

array([ 1838214.89061456,  1195796.43014677,   214003.16585597,
         154145.5921766 ,   102040.01804763,    68731.92840622,
          53790.19925277,    49294.5478562 ,    49814.17612799,
          39916.3129266 ])